In [ ]:
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.BRepGProp import brepgprop_VolumeProperties
from OCC.Core.GProp import GProp_GProps
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
from OCC.Core.gp import gp_Vec, gp_Pnt, gp_Trsf
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform
import sys
import numpy as np
from copy import deepcopy
import ipywidgets as widgets

def load_step_file(filename):
    """Load a STEP file and return the shape."""
    step_reader = STEPControl_Reader()
    status = step_reader.ReadFile(filename)
    if status != IFSelect_RetDone:
        raise Exception("Error: Cannot read STEP file.")
    # Transfer the roots and get the shape
    step_reader.TransferRoots()
    shape = step_reader.OneShape()
    return shape

def compute_mass_properties(shape):
    """Compute mass properties such as volume (interpreted as mass for unit density)
    and the center of mass of the given shape."""
    props = GProp_GProps()
    brepgprop_VolumeProperties(shape, props)
    mass = props.Mass()  # For solids, this gives the volume (mass = volume * density)
    center_of_mass = props.CentreOfMass()
    matrix_of_inertia = props.MatrixOfInertia()
    matrix_of_inertia = np.array([[matrix_of_inertia.Value(1, 1), matrix_of_inertia.Value(1, 2), matrix_of_inertia.Value(1, 3)],
                                  [matrix_of_inertia.Value(2, 1), matrix_of_inertia.Value(2, 2), matrix_of_inertia.Value(2, 3)],
                                  [matrix_of_inertia.Value(3, 1), matrix_of_inertia.Value(3, 2), matrix_of_inertia.Value(3, 3)]])
    return mass, center_of_mass, matrix_of_inertia


In [ ]:
shape_1 = load_step_file("examples/pentagonextrude.step")
shape_2 = load_step_file("examples/hexextrude.step")

shape_1_copy = deepcopy(shape_1)
shape_2_copy = deepcopy(shape_2)

m1, c1, mat1 = compute_mass_properties(shape_1)
m2, c2, mat2 = compute_mass_properties(shape_2)

eig1, v1 = np.linalg.eigh(mat1)
eig2, v2 = np.linalg.eigh(mat2)

s1 = np.sqrt(np.abs(eig1).sum()/m1)
s2 = np.sqrt(np.abs(eig2).sum()/m2)

translation_vector = gp_Vec(-c1.X(), -c1.Y(), -c1.Z())
translation = gp_Trsf()
translation.SetTranslation(translation_vector)
# translation.SetScale(gp_Pnt(0,0,0),1/s1)
shape_1 = BRepBuilderAPI_Transform(shape_1, translation, True).Shape()

scaling = gp_Trsf()
scaling.SetScale(gp_Pnt(0,0,0), 1/s1)
shape_1 = BRepBuilderAPI_Transform(shape_1, scaling, True).Shape()

translation_vector = gp_Vec(-c2.X(), -c2.Y(), -c2.Z())
translation = gp_Trsf()
translation.SetTranslation(translation_vector)
# translation.SetScale(gp_Pnt(0,0,0),1/s2)
shape_2 = BRepBuilderAPI_Transform(shape_2, translation, True).Shape()
scaling = gp_Trsf()
scaling.SetScale(gp_Pnt(0,0,0), 1/s2)
shape_2 = BRepBuilderAPI_Transform(shape_2, scaling, True).Shape()

# rotation to align 2 on 1
R = v1 @ v2.T
rotation = gp_Trsf()
rotation.SetValues(R[0,0], R[0,1], R[0,2], 0,
                    R[1,0], R[1,1], R[1,2], 0,
                    R[2,0], R[2,1], R[2,2], 0)
shape_2 = BRepBuilderAPI_Transform(shape_2, rotation, True).Shape()

# Initialize the Jupyter renderer
display = JupyterRenderer()

# Display the shape interactively in the notebook cell
display.DisplayShape(shape_1)
display.DisplayShape(shape_2, shape_color='red')


display_og = JupyterRenderer()
display_og.DisplayShape(shape_1_copy)
display_og.DisplayShape(shape_2_copy, shape_color='red')


widgets.VBox([display_og.Display(), display.Display()])